In [1]:
import os
import numpy as np
import scipy.stats as sps
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import prune
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from tqdm import tqdm
import pickle
from utils import get_network
from conf import settings
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
import quant_utils
import copy
import random
seed = 0
device = 'cpu'
gpu = 'cuda:1'
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic=True

In [2]:
subset1_mean, subset1_std = settings.CIFAR100_SUBTRAIN_MEAN[0], settings.CIFAR100_SUBTRAIN_STD[0]

def load_model(path, norm=False, dev='cpu'):
    vic = get_network('vgg16', False, num_classes=100).to(dev)
    vic.load_state_dict(torch.load(path, map_location=dev))
    vic.eval()
    if norm:
        return nn.Sequential(transforms.Normalize(subset1_mean, subset1_std), vic)
    return vic


In [3]:
def global_prune(net, p=0.2):
    copy_net = copy.deepcopy(net)
    parameters_to_prune = []
    for name, module in copy_net.named_modules():
        if isinstance(module, torch.nn.Conv2d):
            parameters_to_prune.append((module, 'weight'))
        if isinstance(module, torch.nn.Linear):
            parameters_to_prune.append((module, 'weight'))
    prune.global_unstructured(
        tuple(parameters_to_prune),
        pruning_method=prune.L1Unstructured,
        amount=p,
    )
    return copy_net

# Paths

In [4]:
root_path = '/data1/checkpoint/'
related_folder_path = os.path.join(root_path, 'hash/cifar100/')
unrelated_folder_path = os.path.join(root_path, 'hash/cifar100/independent')

original_path = os.path.join(related_folder_path, 'vgg16_0.pth')
quant_path = os.path.join(related_folder_path, 'vgg16_0_quant.pth')

finetune_path_dict = {}
for fid in range(5):
    folder_name = 'finetune_{}'.format(fid)
    finetune_path_dict[folder_name] = [
        os.path.join(related_folder_path, '{}/finetune_{}.pth'.format(folder_name, i)) for i in range(1, 11)]
    folder_name = 'advfinetune_{}'.format(fid)
    finetune_path_dict[folder_name] = [
        os.path.join(related_folder_path, '{}/finetune_{}.pth'.format(folder_name, i)) for i in range(1, 11)]

for fid in range(5, 10):
    folder_name = 'finetune_{}'.format(fid)
    finetune_path_dict[folder_name] = [
        os.path.join(related_folder_path, '{}/finetune_{}.pth'.format(folder_name, i)) for i in range(1, 21)]
    folder_name = 'advfinetune_{}'.format(fid)
    finetune_path_dict[folder_name] = [
        os.path.join(related_folder_path, '{}/finetune_{}.pth'.format(folder_name, i)) for i in range(1, 21)]

unrelated_path_list = [os.path.join(unrelated_folder_path, 'model_{}.pth'.format(i)) for i in range(200)]

In [5]:
norm = True
n = 1000
input_shape = (n, ) + (3, 32, 32)
# checkpoint_out = {}
output_dict = {}
randf = torch.rand

In [6]:
with torch.no_grad():
    # original net
    original_net = load_model(original_path, norm=norm, dev=gpu)
    output_dict['train'] = original_net(randf(input_shape, device=gpu)).softmax(dim=1).to('cpu')

    # quant net
    net = quant_utils.load_torchscript_model(quant_path, 'cpu')
    if norm:
        net = nn.Sequential(transforms.Normalize(subset1_mean, subset1_std), net)
    output_dict['quant'] = net(randf(input_shape)).softmax(dim=1)
    
    # pruning
    output_dict['prune'] = {}
    for prune_p in [0.8, 0.6, 0.4, 0.2]:
        net = global_prune(original_net, prune_p)
        output_dict['prune'][prune_p] = net(randf(input_shape, device=gpu)).softmax(dim=1).to('cpu')
    
    # finetuned model
    output_dict['finetune'] = {}
    for name, pathlist in tqdm(finetune_path_dict.items()):
        output_dict['finetune'][name] = []
        for path in pathlist:
            net = load_model(path, norm=norm, dev=gpu)
            output_dict['finetune'][name].append(net(randf(input_shape, device=gpu)).softmax(dim=1).to('cpu'))

    # finetuned model
    output_dict['indep'] = []
    for i, path in tqdm(enumerate(unrelated_path_list)):
        net = load_model(path, norm=norm, dev=gpu)
        output_dict['indep'].append(net(randf(input_shape, device=gpu)).softmax(dim=1).to('cpu'))

/home/dt/anaconda3/envs/cvnlp/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448255797/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|██████████| 20/20 [01:15<00:00,  3.78s/it]
200it [01:04,  3.08it/s]


# Train autoencoder

In [7]:

autoencoder = nn.Sequential(
                nn.Linear(100, 64),
                nn.ReLU(),
                nn.Linear(64, 64),
                nn.ReLU(),
                nn.Linear(64, 64),
                nn.ReLU(),
                nn.Linear(64, 100),
            ).to(gpu)
ae_dataset = torch.utils.data.TensorDataset(output_dict['train'], torch.zeros(len(output_dict['train'])).long()) 
ae_train_dataloader = torch.utils.data.DataLoader(ae_dataset, 
                                                  batch_size=128, 
                                                  shuffle=True, num_workers=8)
n_epoch = 10
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=3e-3)
criterion = nn.MSELoss()

with tqdm(total=n_epoch, desc="train") as pbar:
    for epoch in range(n_epoch):
        epoch_loss = 0
        for x, y in ae_train_dataloader:
            x = x.to(gpu)
            outx = autoencoder(x).softmax(dim=1)
            loss = criterion(x, outx)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * len(y)
#         print(epoch, epoch_loss / len(ae_dataset))
        pbar.set_postfix({'loss' : '{0:1.10e}'.format(epoch_loss / len(ae_dataset))}) 
        pbar.update(1)

train: 100%|██████████| 10/10 [00:03<00:00,  2.63it/s, loss=6.2692083347e-10]


In [8]:
autoencoder.to('cpu')

Sequential(
  (0): Linear(in_features=100, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=64, bias=True)
  (5): ReLU()
  (6): Linear(in_features=64, out_features=100, bias=True)
)

In [9]:
vic_errors = []
with torch.no_grad():
    for i in range(1):
        randout = original_net(randf(input_shape, device=gpu)).softmax(dim=1).cpu()
        out = autoencoder(randout).softmax(dim=1)
        vic_errors = torch.sum((randout - out)**2, dim=1).numpy()
        print('{}, {:.2e}, {:.2e}'.format(i, np.mean(vic_errors), np.std(vic_errors)))

0, 6.14e-08, 5.97e-08


In [10]:
with torch.no_grad():
    out = autoencoder(output_dict['train']).softmax(dim=1)
    errors = torch.sum((output_dict['train'] - out)**2, dim=1).numpy()
    print(sps.ks_2samp(errors, vic_errors))
    print("{:.2e}, {:.2e}".format(np.mean(errors), np.std(errors)))

KstestResult(statistic=0.036, pvalue=0.5362766985932181)
6.27e-08, 5.73e-08


In [11]:
with torch.no_grad():
    
    out = autoencoder(output_dict['quant']).softmax(dim=1)
    errors = torch.sum((output_dict['quant'] - out)**2, dim=1).numpy()
    print(sps.ks_2samp(errors, vic_errors))
    print("{:.2e}, {:.2e}".format(np.mean(errors), np.std(errors)))

KstestResult(statistic=0.021, pvalue=0.9802627322900355)
6.20e-08, 5.99e-08


In [12]:
with torch.no_grad():
    for k, v in output_dict['prune'].items():
        print(k)
        out = autoencoder(v).softmax(dim=1)
        errors = torch.sum((v - out)**2, dim=1).numpy()
        stats, pv = sps.ks_2samp(errors, vic_errors)
        print("{:.2e}, {:.2e}, {:.2e}, {:.2e}".format(
            np.mean(errors), np.std(errors), stats, pv))
        

0.8
2.91e-02, 7.80e-03, 1.00e+00, 0.00e+00
0.6
1.81e-07, 1.31e-07, 6.50e-01, 1.41e-199
0.4
6.02e-08, 5.25e-08, 3.00e-02, 7.59e-01
0.2
6.39e-08, 6.96e-08, 5.30e-02, 1.21e-01


In [13]:
finetune_errors = {}
indep_errors = []

In [14]:
with torch.no_grad():
    for k, vlist in output_dict['finetune'].items():
        print(k)
        finetune_errors[k] = []
        for i, v in enumerate(vlist):
            out = autoencoder(v).softmax(dim=1)
            errors = torch.sum((v - out)**2, dim=1).numpy()
            stats, pv = sps.ks_2samp(errors, vic_errors)
            finetune_errors[k].append(errors)
            print(i, "{:.2e}, {:.2e}, {:.2e}, {:.2e}".format(
            np.mean(errors), np.std(errors), stats, pv))

finetune_0
0 2.01e-08, 1.76e-08, 5.80e-01, 5.59e-156
1 2.44e-08, 2.00e-08, 4.77e-01, 2.55e-103
2 3.57e-08, 2.90e-08, 3.01e-01, 2.28e-40
3 2.90e-08, 2.62e-08, 4.12e-01, 2.40e-76
4 1.61e-08, 1.32e-08, 6.66e-01, 1.54e-210
5 2.21e-08, 2.31e-08, 5.59e-01, 3.45e-144
6 1.83e-08, 1.64e-08, 6.25e-01, 3.24e-183
7 2.27e-08, 2.29e-08, 5.23e-01, 2.92e-125
8 2.09e-08, 2.05e-08, 5.73e-01, 5.50e-152
9 2.05e-08, 1.54e-08, 5.51e-01, 7.62e-140
advfinetune_0
0 3.57e-08, 1.12e-07, 4.00e-01, 7.39e-72
1 3.45e-08, 3.21e-08, 3.25e-01, 4.07e-47
2 3.29e-08, 5.23e-08, 4.16e-01, 7.11e-78
3 5.92e-08, 7.62e-08, 1.25e-01, 3.17e-07
4 3.22e-08, 3.10e-08, 3.56e-01, 1.16e-56
5 3.72e-08, 5.45e-08, 3.64e-01, 2.79e-59
6 6.03e-08, 9.17e-08, 1.74e-01, 1.24e-13
7 5.20e-08, 7.96e-08, 2.16e-01, 7.73e-21
8 5.60e-08, 6.60e-08, 7.40e-02, 8.35e-03
9 6.47e-08, 8.38e-08, 1.20e-01, 1.08e-06
finetune_1
0 2.19e-08, 2.34e-08, 5.41e-01, 1.59e-134
1 3.65e-08, 3.39e-08, 3.11e-01, 4.10e-43
2 2.74e-08, 2.21e-08, 4.27e-01, 3.62e-82
3 1.91e-08, 

In [15]:
with torch.no_grad():
    for i, v in enumerate(output_dict['indep']):
        out = autoencoder(v).softmax(dim=1)
        errors = torch.sum((v - out)**2, dim=1).numpy()
        indep_errors.append(errors)
        stats, pv = sps.ks_2samp(errors, vic_errors)
        print(i, "{:.2e}, {:.2e}, {:.2e}, {:.2e}".format(
            np.mean(errors), np.std(errors), stats, pv))

0 1.23e+00, 4.07e-01, 1.00e+00, 0.00e+00
1 4.51e-07, 6.31e-06, 1.97e-01, 2.21e-17
2 9.81e-03, 9.65e-02, 8.75e-01, 0.00e+00
3 5.67e-03, 7.61e-02, 7.72e-01, 4.08e-294
4 1.01e-01, 2.00e-01, 1.00e+00, 0.00e+00
5 8.24e-05, 1.10e-03, 7.86e-01, 8.73e-307
6 1.13e-04, 1.14e-03, 9.26e-01, 0.00e+00
7 8.05e-08, 2.17e-07, 1.44e-01, 1.86e-09
8 2.43e-01, 3.02e-01, 1.00e+00, 0.00e+00
9 2.24e-02, 1.42e-01, 7.72e-01, 4.08e-294
10 4.18e-02, 1.61e-01, 9.92e-01, 0.00e+00
11 1.92e-01, 3.76e-01, 9.94e-01, 0.00e+00
12 4.53e-05, 1.31e-03, 2.45e-01, 9.52e-27
13 2.02e-05, 4.21e-04, 5.67e-01, 1.30e-148
14 1.75e-01, 3.72e-01, 9.97e-01, 0.00e+00
15 3.86e-06, 7.34e-05, 2.60e-01, 4.15e-30
16 4.45e-01, 5.62e-01, 9.99e-01, 0.00e+00
17 1.89e-02, 1.12e-01, 8.88e-01, 0.00e+00
18 1.38e+00, 6.29e-01, 1.00e+00, 0.00e+00
19 6.00e-01, 5.89e-01, 9.99e-01, 0.00e+00
20 3.83e-01, 5.31e-01, 9.93e-01, 0.00e+00
21 1.32e-01, 3.44e-01, 9.90e-01, 0.00e+00
22 1.65e-02, 9.26e-02, 9.38e-01, 0.00e+00
23 1.15e-07, 1.92e-06, 7.11e-01, 1.46e-2

In [16]:
torch.save(autoencoder.state_dict(), '../results/hash/cifar100/autoencoder.pt')
pickle.dump((output_dict, finetune_errors, indep_errors), open("../results/hash/cifar100/output_dict.pkl", "wb"))


In [17]:
finetune_mean_errors = []
for k, vlist in finetune_errors.items():
    if int(k.split('_')[-1])<5:
        for v in vlist:
            finetune_mean_errors.append(np.mean(v))
print(np.max(finetune_mean_errors))

1.0050122e-07


In [18]:
test_finetune_mean_errors = []
for k, vlist in finetune_errors.items():
    if int(k.split('_')[-1])>4:
        for v in vlist:
            
            test_finetune_mean_errors.append(np.mean(v))

In [19]:
# True positive
test_finetune_mean_errors = np.array(test_finetune_mean_errors)
print(len(test_finetune_mean_errors[test_finetune_mean_errors<=np.max(finetune_mean_errors)]))

191


In [20]:
test_finetune_mean_errors = np.array(test_finetune_mean_errors)
print(len(test_finetune_mean_errors[test_finetune_mean_errors>np.max(finetune_mean_errors)]))

9


In [21]:
test_indep_mean_errors = []
for v in indep_errors:
    test_indep_mean_errors.append(np.mean(v))
test_indep_mean_errors = np.array(test_indep_mean_errors)
print("True N\n", len(test_indep_mean_errors[test_indep_mean_errors>np.max(finetune_mean_errors)]))
print("False P\n", len(test_indep_mean_errors[test_indep_mean_errors<np.max(finetune_mean_errors)]))

True N
 186
False P
 14


In [22]:
# from utils import compute_accuracy, get_test_dataloader_cifar100
# test_loader = get_test_dataloader_cifar100((0, 0, 0), (1, 1, 1))
# acc, _, _ = compute_accuracy(quant_net, test_loader, 'cpu')
# accquant, _, _ = compute_accuracy(quant_net, test_loader, 'cpu')
# acc20, _, _ = compute_accuracy(pruned_model_2, test_loader, 'cpu')
# acc40, _, _ = compute_accuracy(pruned_model_4, test_loader, 'cpu')
# acc60, _, _ = compute_accuracy(pruned_model_6, test_loader, 'cpu')
# acc80, _, _ = compute_accuracy(pruned_model_8, test_loader, 'cpu')
# print(acc, accquant, acc20, acc40, acc60, acc80)
# 0.6387 0.6387 0.6395 0.6395 0.6405 0.5619